# Austin Crime Project 

___

By: Alejandro Garcia, Matthew Luna, Kristofer Rivera, Oliver Ton
<br>
Date: June 2022

___

## Project Goals



___

## Project Description



___

## Importing the Required Modules

Everything we need to run the code blocks in this notebook are located in the top level directory. To run the code blocks in this report you will need numpy, pandas, matplotlib, seaborn, and sklearn installed on your computer.

___

## Data Acquisition and Preparation



___

## Exploratory Analysis

(Project specific details here)

In the visualizations below we aim to answer some questions about the data. Details about how these visualization are created can be found in the explore.py file.

### Question 1

### Question 2

### Question 3

### Question 4

### Key Takeaways

___

## Modeling



___

## Key Takeaways and Recommendations



___

## Next Steps

